<a href="https://colab.research.google.com/github/seanbenhur/papers/blob/main/Lenet/Lenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LeNet Paper Implementation
LeNet is a 7-level convolutional network by LeCun in 1998 that classifies digits and used by several banks to recognise hand-written numbers on cheques digitized in 32x32 pixel greyscale input images. 

In [66]:
#import library
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torch.nn.functional as F
import torch.optim as optim

In [67]:
transform = transforms.Compose([transforms.Resize(32),
                                transforms.ToTensor(),
                                ])
train_ds = MNIST(root='data/',train=True,download=True,transform=transform)
test_ds = MNIST(root='data/',train=False,download=False,transform=transform)

batch_size=128
train_loader = DataLoader(train_ds,batch_size,shuffle=True,num_workers=4,pin_memory=True)
val_loader = DataLoader(test_ds,batch_size,num_workers=4,pin_memory=True)

In [68]:
def accuracy(outputs,labels):
  _,preds = torch.max(outputs,dim=1)
  return torch.tensor(torch.sum(preds==labels).item()/len(preds))
 
class ImageClassificationBase(nn.Module):
  def training_step(self,batch):
    images, labels = batch
    out = self(images)
    loss = F.cross_entropy(out,labels)
    return loss
  
  def validation_step(self,batch):
    images, labels = batch
    out = self(images)
    loss = F.cross_entropy(out,labels)
    acc = accuracy(out,labels)
    return {'val_loss': loss.detach(),'val_acc': acc}
  
  def validation_epoch_end(self,outputs):
    batch_losses = [x['val_loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()
    batch_accs = [x['val_acc'] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()
    return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
  
  def epoch_end(self, epoch, result):
    print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

# Lenet Architecture

In [69]:
class Lenet(ImageClassificationBase):
  def __init__(self):
    super(Lenet,self).__init__()
    self.tanh = nn.Tanh()      
    self.pool = nn.AvgPool2d(kernel_size=(2,2),stride=(2,2))
    self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=(5,5),stride=(1,1))
    self.conv2 = nn.Conv2d(in_channels=6,out_channels=16,kernel_size=(5,5),stride=(1,1))
    self.conv3 = nn.Conv2d(in_channels=16,out_channels=120,kernel_size=(5,5),stride=(1,1))
    self.linear1 = nn.Linear(120,84)
    self.linear2 = nn.Linear(84,10)

  def forward(self,x):
    x = self.tanh(self.conv1(x))
    x = self.pool(x)
    x = self.tanh(self.conv2(x))
    x = self.pool(x)
    x = self.tanh(self.conv3(x))
    x = x.reshape(x.shape[0],-1)
    x = self.tanh(self.linear1(x))
    x = self.linear2(x)
    return x

model = Lenet()

In [70]:
def get_default_device():
  """Pick GPU if available else CPU"""
  if torch.cuda.is_available():
    return torch.device('cuda')
  else:
    return torch.device('cpu')

In [71]:
device = get_default_device()
device

device(type='cuda')

In [72]:

def to_device(data,device):
  """Move tensors to chosen device"""
  if isinstance(data,(list,tuple)):
    return [to_device(x,device) for x in data]
  return data.to(device,non_blocking=True)

In [73]:
class DeviceDataLoader():
  """Wrap a DataLoader to move data to a device"""
  def __init__(self,dl,device):
    self.dl = dl
    self.device =  device
  def __iter__(self):
    """Yield a batch of data to a dataloader"""
    for b in self.dl:
      yield to_device(b, self.device)
  def __len__(self):
    """Number of batches"""
    return len(self.dl)

In [74]:
train_loader = DeviceDataLoader(train_loader,device)
val_loader = DeviceDataLoader(val_loader,device)
model = Lenet()
to_device(model,device)

Lenet(
  (tanh): Tanh()
  (pool): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (linear1): Linear(in_features=120, out_features=84, bias=True)
  (linear2): Linear(in_features=84, out_features=10, bias=True)
)

In [75]:
 @torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)
 
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    train_losses =[]
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [76]:
history = [evaluate(model, val_loader)]
history

[{'val_acc': 0.11323180049657822, 'val_loss': 2.301053762435913}]

In [77]:
history = fit(3,000.1,model,train_loader,val_loader)

Epoch [0], train_loss: 0.6566, val_loss: 0.2454, val_acc: 0.9258
Epoch [1], train_loss: 0.4271, val_loss: 0.1334, val_acc: 0.9598
Epoch [2], train_loss: 0.3255, val_loss: 0.0946, val_acc: 0.9721
